In [33]:
import pandas as pd
import sqlite3
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [34]:
abilities = pd.read_csv('processed_data/ability.csv', index_col=0)
combat = pd.read_csv('processed_data/combat.csv', index_col=0)
pokemon_abilities = pd.read_csv('processed_data/pokemon_ability.csv', index_col=0)
pokemon = pd.read_csv('processed_data/pokemon.csv', index_col=0)
trainers_pokemon = pd.read_csv('processed_data/trainer_pokemon.csv', index_col=0)
trainers = pd.read_csv('processed_data/trainer.csv', index_col=0)
types = pd.read_csv('processed_data/type.csv', index_col=0)

In [35]:
# make fact table
fact_table = trainers_pokemon[['trainer_id', 'pokemon_id']]

win_abilities = []
types_set = []
win_counts = []
loss_counts = []
for index, row in fact_table.iterrows():
    wins = combat.query("(winning_pokemon_id == " + str(row['pokemon_id']) + ") & (winning_trainer_id == " + str(row['trainer_id']) + ")")
    losses = combat.query("(losing_pokemon_id == " + str(row['pokemon_id']) + ") & (losing_trainer_id == " + str(row['trainer_id']) + ")")
    win_counts.append(len(wins))
    loss_counts.append(len(losses))
    ability_freq = {}
    for i, r in wins.iterrows():
        if r['winning_ability'] in ability_freq.keys():
            ability_freq[r['winning_ability']] = ability_freq[r['winning_ability']] + 1
        else:
            ability_freq[r['winning_ability']] = 1
    
    max_val = -1
    max_id = -1
    for k, v in ability_freq.items():
        if v > max_val:
            max_val = v
            max_id = k
    win_abilities.append(max_id)
    
    type_id = pokemon.query('pokemon_id == ' + str(row['pokemon_id']))['type_id'].iloc[0]
    types_set.append(type_id)

fact_table['winning_ability_id'] = win_abilities
fact_table['type_id'] = types_set
fact_table['wins'] = win_counts
fact_table['losses'] = loss_counts

fact_table.index = fact_table.index.rename('fact_table_id')

fact_table

trainer_id  pokemon_id  winning_ability_id  type_id  wins  \
fact_table_id                                                              
1                       1         770               144.0       66    15   
2                       1         778                36.0       67    17   
3                       2          94                27.0       61     4   
4                       3         637               184.0        9     1   
5                       4         393                35.0      148    12   
6                       4         394                35.0      148    26   
7                       4         395                35.0      161     1   
8                       5         518               190.0      117    19   
9                       6          77                47.0       46     7   
10                      6          78                -1.0       46     0   
11                      7          18                13.0      100     5   
12                      7           9               128.0      155    24   
13                      8         296               153.0       38    11   
14                      8         297               153.0       38     2   
15                      9         252               201.0       71     6   
16                      9         253               201.0       71     9   
17                      9         254               201.0       71    22   
18                     10          95               211.0        5    12   
19                     10         248               202.0      129    17   
20                     11          68               174.0       40    13   
21                     12         615                87.0       94    10   
22                     13          15               161.0        3     3   
23                     14         542               110.0        3    27   
24                     15         579               133.0      117    10   
25                     16         677               112.0      117    23   
26                     16          16                -1.0      100     0   
27                     17         460               167.0       74     8   
28                     18         508               147.0      100    12   
29                     19         242                61.0      100    16   
30                     20         230                29.0      158    12   
31                     21         536               207.0      156    12   
32                     21         537               207.0      156    18   
33                     22         695               164.0       34     3   
34                     23         419                -1.0      148     0   
35                     24         445               138.0       24     7   
36                     24         784               110.0       28     6   
37                     25         484               190.0      158    10   
38                     26         387               154.0       71    11   
39                     26         388               154.0       71    12   
40                     26         389               154.0      165     4   
41                     26         869                -1.0       36     0   
42                     27         282               190.0      118     5   
43                     28         373               102.0       23     4   
44                     29         523               145.0       29    13   
45                     29         479                -1.0       33     0   
46                     30          45                42.0       70     2   
47                     30         764               105.0       36     8   
48                     31         155                48.0       46    23   
49                     31         156                48.0       46     7   
50                     31         157                48.0       46    17   
51                     32         429                87.0      

In [36]:
combat_adb = combat[['winning_pokemon_id', 'winning_trainer_id', 'losing_pokemon_id', 'losing_trainer_id']]
combat_adb

winning_pokemon_id  winning_trainer_id  losing_pokemon_id  \
combat_id                                                              
1                         283                 113                 15   
2                         151                  91                 87   
3                         448                  55                763   
4                         242                  19                579   
5                         130                  57                319   
6                         612                  47                579   
7                          71                  95                777   
8                         445                  24                 45   
9                         154                  68                698   
10                        307                  76                407   
11                        545                 100                 28   
12                        544                 100                324   
13                        795                  67                437   
14                        337                  77                770   
15                        635                  47                158   
16                        159                  56                152   
17                        350                  79                160   
18                        159                  56                 77   
19                          9                   7                484   
20                        248                  10                750   
21                         94                   2                695   
22                        409                  97                254   
23                        336                  66                698   
24                        409                  97                376   
25                        149                  57                545   
26                        362                  37                 68   
27                        713                 121                460   
28                        699                  40                384   
29                         65                 101                644   
30                        502                  45                 86   
31                        498                  46                282   
32                         59                  71                 87   
33                        695                  22                637   
34                        394                   4                297   
35                        429                  32                475   
36                        579                  15                373   
37                        158                  56                258   
38                        248                  10                743   
39                        716                  92                565   
40                         68                  11                693   
41                        307                  76                149   
42                        254                   9                283   
43                        545                 100                241   
44                        542                  14                615   
45                        496                  98                319   
46                        336                  66                283   
47                         65                 101                523   
48                        717                  69                150   
49                        518                   5                101   
50                        581                 107                299   
51                        778                   1                334   
52                        716                  92                296   
53                        764                  30                445   
54                        157                  31  

In [37]:
trainer_adb = trainers[['trainer_name']]
trainer_adb

trainer_name
trainer_id                    
1                      Acerola
2                       Agatha
3                        Alder
4                        Barry
5                       Bianca
6                       Blaine
7                         Blue
8                       Brawly
9                      Brendan
10                       Brock
11                       Bruno
12                      Brycen
13                       Bugsy
14                       Burgh
15                     Caitlin
16                       Calem
17                     Candice
18                      Cheren
19                      Cheryl
20                       Clair
21                        Clay
22                     Clemont
23                Crasher Wake
24                     Cynthia
25                       Cyrus
26                        Dawn
27                     Diantha
28                       Drake
29                       Elesa
30                       Erika
31                       Ethan
32                     Fantina
33                    Flannery
34                       Flint
35                    Gardenia
36                    Giovanni
37                      Glacia
38                     Gladion
39                      Gloria
40                       Grant
41                    Grimsley
42                       Guzma
43                        Hapu
44                         Hau
45                     Hilbert
46                       Hilda
47                        Iris
48                       James
49                      Janine
50                     Jasmine
51                      Jessie
52                      Kahili
53                       Karen
54                        Koga
55                     Korrina
56                        Kris
57                       Lance
58                        Leaf
59                        Lear
60                        Leon
61                      Lillie
62                       Lisia
63                        Liza
64                     Lorelei
65                   Lt. Surge
66                        Lucy
67                    Lusamine
68                        Lyra
69                    Lysandre
70                      Mallow
71                      Marley
72                      Marlon
73                      Marnie
74                     Marshal
75                         May
76                     Maylene
77                        Mina
78                       Misty
79                     Molayne
80                       Morty
81                           N
82                        Nanu
83                        Nate
84                      Noland
85                      Norman
86                      Olivia
87                      Phoebe
88                       Piers
89                    Plumeria
90             Professor Kukui
91               Professor Oak
92          Professor Sycamore
93                       Pryce
94                      Rachel
95                       Ramos
96                         Red
97                       Roark
98                        Rosa
99                     Roxanne
100                      Roxie
101                    Sabrina
102                     Sawyer
103                     Serena
104                   Shauntal
105                    Siebold
106                     Silver
107                      Skyla
108                  Sophocles
109                     Steven
110                       Tate
111                    Thorton
112                    Valerie
113                      Viola
114                    Volkner
115                    Wallace
116                      Wally
117                    Whitney
118                   Wikstrom
119                       Will
120                     Winona
121                    Wulfric
122                     Zinnia

In [38]:
pokemon_adb = pokemon[['pokemon_name', 'total_points', 'HP', 'attack', 'defense', 'special_attack', 'special_defense', 'speed', 'family']]
pokemon_adb

pokemon_name  total_points   HP  attack  defense  special_attack  \
pokemon_id                                                                     
1              Bulbasaur           318   45      49       49              65   
2                Ivysaur           405   60      62       63              80   
3               Venusaur           525   80      82       83             100   
4             Charmander           309   39      52       43              60   
5             Charmeleon           405   58      64       58              80   
6              Charizard           534   78      84       78             109   
7               Squirtle           314   44      48       65              50   
8              Wartortle           405   59      63       80              65   
9              Blastoise           530   79      83      100              85   
10              Caterpie           195   45      30       35              20   
11               Metapod           205   50      20       55              25   
12            Butterfree           395   60      45       50              90   
13                Weedle           195   40      35       30              20   
14                Kakuna           205   45      25       50              25   
15              Beedrill           395   65      90       40              45   
16                Pidgey           251   40      45       40              35   
17             Pidgeotto           349   63      60       55              50   
18               Pidgeot           479   83      80       75              70   
19               Rattata           253   30      56       35              25   
20              Raticate           413   55      81       60              50   
21               Spearow           262   40      60       30              31   
22                Fearow           442   65      90       65              61   
23                 Ekans           288   35      60       44              40   
24                 Arbok           448   60      95       69              65   
25               Pikachu           320   35      55       40              50   
26                Raichu           485   60      90       55              90   
27             Sandshrew           300   50      75       85              20   
28             Sandslash           450   75     100      110              45   
29               Nidoran           275   55      47       52              40   
30              Nidorina           365   70      62       67              55   
31             Nidoqueen           505   90      92       87              75   
32               Nidoran           273   46      57       40              40   
33              Nidorino           365   61      72       57              55   
34              Nidoking           505   81     102       77              85   
35              Clefairy           323   70      45       48              60   
36              Clefable           483   95      70       73              95   
37                Vulpix           299   38      41       40              50   
38             Ninetales           505   73      76       75              81   
39            Jigglypuff           270  115      45       20              45   
40            Wigglytuff           435  140      70       45              85   
41                 Zubat           245   40      45       35              30   
42                Golbat           455   75      80       70              65   
43                Oddish           320   45      50       55              75   
44                 Gloom           395   60      65       70              85   
45             Vileplume           490   75      80       85             110   
46                 Paras           285   35      70       55              45   
47              Parasect           405   60      95       80              60   
48               Venonat           305   60      55       50              40   
49          

In [39]:
types_adb = types[['type_1', 'type_2']]
types_adb

type_1    type_2
type_id                    
0             bug       NaN
1             bug    flying
2             bug    poison
3             bug     grass
4             bug     steel
5             bug      rock
6             bug    ground
7             bug     ghost
8             bug  electric
9             bug      fire
10            bug     fairy
11            bug     water
12           dark       NaN
13           dark    flying
14           dark       ice
15           dark      fire
16           dark     ghost
17           dark  fighting
18           dark     steel
19           dark    dragon
20           dark   psychic
21         dragon       NaN
22         dragon   psychic
23         dragon    flying
24         dragon    ground
25         dragon      fire
26         dragon  electric
27         dragon       ice
28         dragon  fighting
29       electric       NaN
30       electric  electric
31       electric     steel
32       electric    flying
33       electric     ghost
34       electric    normal
35       electric     fairy
36          fairy       NaN
37          fairy    flying
38       fighting       NaN
39       fighting   psychic
40       fighting     steel
42       fighting    flying
43       fighting       ice
44           fire       NaN
45           fire    flying
46           fire       ice
47           fire      rock
48           fire  fighting
49           fire    ground
50           fire     steel
51           fire      fire
52           fire   psychic
53           fire    normal
54           fire     water
57         flying       NaN
59          ghost    poison
60          ghost       NaN
61          ghost    flying
63          ghost    dragon
64          ghost      fire
65          ghost     grass
66          ghost    ground
67          ghost     fairy
68          grass    poison
69          grass   psychic
70          grass    flying
71          grass       NaN
72          grass      dark
73          grass  fighting
74          grass       ice
75          grass     grass
76          grass     fairy
77          grass     steel
79         ground       ice
80         ground    ground
81         ground       NaN
83         ground      rock
84         ground    flying
86         ground   psychic
87         ground     steel
88         ground      dark
89         ground  electric
91            ice   psychic
92            ice    flying
94            ice       NaN
95            ice     water
96            ice     ghost
97         normal    flying
98         normal      dark
99         normal     fairy
100        normal       NaN
101        normal   psychic
102        normal     water
103        normal     grass
104        normal    ground
105        normal  fighting
106        normal    dragon
107        poison       NaN
108        poison    ground
109        poison    flying
110        poison    poison
111        poison      dark
113        poison  fighting
114        poison     water
115        poison    dragon
116        poison      fire
117       psychic       NaN
118       psychic     fairy
119       psychic    flying
123       psychic     ghost
124       psychic     steel
126          rock     water
127          rock    flying
128          rock       NaN
129          rock      dark
130          rock   psychic
131          rock     grass
133          rock     steel
134          rock  fighting
135          rock    dragon
136          rock       ice
137          rock     fairy
138          rock    poison
140         steel    flying
141         steel     fairy
144         steel       NaN
145         steel    dragon
147         steel     ghost
148         water       NaN
149         water  fighting
151         water   psychic
153         water    flying
154         water  electric
155         water     fairy
156         water    ground
158         water    dragon
159         water     grass
160         water      dark
161         water     steel
162         water     ghost
164           bug  fighting
165    

In [41]:
ability_adb = abilities[['ability_name']]
ability_adb

ability_name
ability_id                  
1               Adaptability
2                  Aftermath
3                   Air Lock
4                Anger Point
5               Anticipation
6                 Arena Trap
7                 Bad Dreams
8                 Ball Fetch
9                    Battery
10              Battle Armor
11               Beast Boost
12                   Berserk
13                 Big Pecks
14                     Blaze
15               Bulletproof
16               Cheek Pouch
17               Chlorophyll
18                Clear Body
19                Cloud Nine
20              Color Change
21                  Comatose
22               Competitive
23             Compound Eyes
24                  Contrary
25                 Corrosion
26               Cotton Down
27               Cursed Body
28                Cute Charm
29                      Damp
30                    Dancer
31                 Dark Aura
32          Dauntless Shield
33                  Dazzling
34                 Defeatist
35                   Defiant
36                  Disguise
37                  Download
38                   Drizzle
39                   Drought
40                  Dry Skin
41                Early Bird
42              Effect Spore
43            Electric Surge
44            Emergency Exit
45                Fairy Aura
46                    Filter
47                Flame Body
48                Flash Fire
49               Flower Gift
50               Flower Veil
51                    Fluffy
52                  Forecast
53                  Forewarn
54                     Frisk
55           Full Metal Body
56                  Fur Coat
57                  Gluttony
58              Grassy Surge
59              Gulp Missile
60                      Guts
61                    Healer
62                 Heatproof
63               Heavy Metal
64              Honey Gather
65                Huge Power
66             Hunger Switch
67                    Hustle
68                 Hydration
69              Hyper Cutter
70                  Ice Body
71                  Ice Face
72                Illuminate
73                  Illusion
74                  Immunity
75               Infiltrator
76               Innards Out
77               Inner Focus
78                  Insomnia
79                Intimidate
80            Intrepid Sword
81                Iron Barbs
82                 Iron Fist
83                 Justified
84                  Keen Eye
85                     Klutz
86                Leaf Guard
87                  Levitate
88               Light Metal
89             Lightning Rod
90                    Limber
91               Liquid Ooze
92               Magic Guard
93                  Magician
94               Magma Armor
95               Magnet Pull
96              Marvel Scale
97                 Merciless
98                     Minus
99               Misty Surge
100             Mold Breaker
101              Motor Drive
102                    Moxie
103                Multitype
104                    Mummy
105             Natural Cure
106         Neutralizing Gas
107                 No Guard
108                Normalize
109                Oblivious
110                 Overcoat
111                 Overgrow
112                Own Tempo
113                   Pickup
114                     Plus
115              Poison Heal
116             Poison Point
117             Poison Touch
118          Power Construct
119               Power Spot
120                Prankster
121                 Pressure
122              Prism Armor
123            Psychic Surge
124                Punk Rock
125               Pure Power
126          Queenly Majesty
127               Quick Feet
128                Rain Dish
129                  Rattled
130                 Receiver
131                 Reckless
132              Refrigerate
133              Regenerator
134                    Ripen
135                  Rivalry
136               RKS System
137           

In [42]:
fact_table.to_csv('ADB/fact_table.csv')
combat_adb.to_csv('ADB/combat_adb.csv')
trainer_adb.to_csv('ADB/trainer_adb.csv')
pokemon_adb.to_csv('ADB/pokemon_adb.csv')
types_adb.to_csv('ADB/types_adb.csv')
ability_adb.to_csv('ADB/ability_adb.csv')

In [46]:
def connect_sqlite():
    con = sqlite3.connect('../data.db')
    return con, con.cursor()

In [47]:
con, cur = connect_sqlite()

fact_table.to_sql("fact_table", con, if_exists='replace')
combat_adb.to_sql("combat_adb", con, if_exists='replace')
trainer_adb.to_sql("trainer_adb", con, if_exists='replace')
pokemon_adb.to_sql("pokemon_adb", con, if_exists='replace')
types_adb.to_sql("types_adb", con, if_exists='replace')
ability_adb.to_sql("ability_adb", con, if_exists='replace')